In [ ]:
# Nicole's code starts here 
# Importing "The Numbers" data & cleaning it up

In [ ]:
import pandas as pd
import numpy as np
from config import OMB_api_key
import requests
import json
import re
from pprint import pprint

In [ ]:
numbers_df = pd.read_csv('DataFiles/TheNumbers_Original.csv')
print(numbers_df.shape)
numbers_df.head(1)

In [ ]:
# Create month released column & add to dataframe. Probably don't need to do this since we can
# return the month after turning it into a datetime data type
numbers_df['Domestic Release Date'] = numbers_df['Domestic Release Date'].astype('datetime64[ns]')
numbers_df['Worldwide Release Date'] = numbers_df['Worldwide Release Date'].astype('datetime64[ns]')
month = pd.DatetimeIndex(numbers_df['Domestic Release Date']).month
numbers_df.insert(3, 'Month Released (Domestic)', month)
numbers_df.head(1)

In [ ]:
# Convert columns 11 + to integer
numbers_df[numbers_df.columns[11:]] = numbers_df[numbers_df.columns[11:]].apply\
(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)

In [ ]:
# Adding available oscar count per year. Somebody please check these calculations if we use this!
numbers_df['Total Oscars Awarded in Year'] = ''
for index, row in numbers_df.iterrows():
    year = row['Year Released (Domestic)']
    if year == 1980:
        numbers_df.loc[index, 'Total Oscars Awarded in Year'] = 22
    elif year in range(1981,1995) or year == 1999:
        numbers_df.loc[index, 'Total Oscars Awarded in Year'] = 23
    elif year in range(2001,2020):
        numbers_df.loc[index, 'Total Oscars Awarded in Year'] = 25
    else:
        numbers_df.loc[index, 'Total Oscars Awarded in Year'] = 24

In [ ]:
# Title needs cleaning (remove apostrophes, colons, eplipses, "Ep. xxx:")
# The order it is replaced is sequential (ie: relacing Ep. I, followd by Ep. II returns I)

# Creating new title column so we can use original title later
numbers_df.insert(6, 'Query_Title', numbers_df['Title'])

# Replacing characters
numbers_df[numbers_df.columns[6:7]] = numbers_df[numbers_df.columns[6:7]].apply\
(lambda x: x.str.replace(":",'')).apply(lambda x: x.str.replace("Ep.","Episode"))

In [ ]:
# Limit movies to 40 per year - defined as top 40 by adjusted gross ***
numbers_df = numbers_df.sort_values(['Year Released (Domestic)', 'Infl. Adj. Dom. Box Office'],
                                    ascending = [True, False])
numbers_df = numbers_df.reset_index(drop = True)

In [ ]:
# Still limiting...
numbers_df['Year Index'] = ''
year_compare = 1980
count = 0
for index, row in numbers_df.iterrows():
    year = row['Year Released (Domestic)']
    if year == year_compare:
        count += 1
        numbers_df.loc[index, 'Year Index'] = count
    else:
        count = 1
        numbers_df.loc[index, 'Year Index'] = count
        year_compare += 1

In [ ]:
# ... a little more & voila!
top_40_df = numbers_df.loc[(numbers_df['Year Index'] <=40), ['Title', 'Query_Title',
                                                             'Domestic Release Date',
                                                             'Year Released (Domestic)',
                                                             'Month Released (Domestic)',
                                                             'Infl. Adj. Dom. Box Office',
                                                             'Domestic Box Office',
                                                             'Genre', 'Theatrical Distributor',
                                                             'Total Oscars Awarded in Year']]
top_40_df = top_40_df.sort_values('Infl. Adj. Dom. Box Office', ascending = False)
top_40_df = top_40_df.reset_index(drop = True)
top_40_df.to_csv('DataFiles/TheNumbers_Cleaned.csv')
print(top_40_df.shape)
top_40_df.head(1)

In [ ]:
# *********  This is the end of data_cleaning & start of request tests ***********

In [ ]:
# REQUEST TESTING (Okay to remove cell)
# Sample JSON in case you want to run one specific movie title
# movie_title = "The battle of the five armies"
# params = {'type': 'movie', 'apikey': OMB_api_key, 't': movie_title}
# url = 'http://www.omdbapi.com/?t='
# response = requests.get(url, params).json()
# pprint(response)

In [ ]:
# REQUEST TESTING (Okay to remove cell)
# Subset dataframe for request testing
# test_subset = omdb_df.iloc[25:39, :]

In [ ]:
# REQUEST TESTING (Okay to remove cell)
# Testing Requests on subset.
# If one of the values within a found movie is missing, it stops inputting data into DF after that
#     and I told it to print that so we know (uncomment the metascore row to view this)
# If we find a lot of missing movies, we could look into adding a year parameter.
#     It looks like it returns the first movie found (ie: 'Star Wars' returns 'Star Wars IV')

# params = {"type": "movie", "apikey": OMB_api_key}
# url = "http://www.omdbapi.com/?t="
# count = 0
# for index, row in test_subset.iterrows():
#     params["t"] = row["Query_Title"]
#     response = requests.get(url, params).json()
#     if response['Response'] == 'True':
#         try:
#             omdb_df.loc[index, 'Awards'] = response['Awards']
#             omdb_df.loc[index, 'Metascore'] = response['Metascore']
#             omdb_df.loc[index, 'IMDB'] = response['imdbRating']
#             omdb_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
#             omdb_df.loc[index, 'Rated'] = response['Rated']
#             omdb_df.loc[index, 'Director'] = response['Director']
#             omdb_df.loc[index, 'Runtime'] = response['Runtime']
#             omdb_df.loc[index, 'Country'] = response['Country']
#         except:
#             print(f'{row.Query_Title.upper()} (row {count}) has missing data')
#         count += 1
#     else:
#         print(f'{row.Query_Title.upper()} (row {count}) was not found')
#         count += 1

In [ ]:
# test_subset.head()

In [ ]:
# ***** API requests All Data *****

# params = {"type": "movie", "apikey": OMB_api_key}
# url = "http://www.omdbapi.com/?t="
# count = 0
# for index, row in omdb_df.iterrows():
#     params['t'] = row["Query_Title"]
#     response = requests.get(url, params).json()
#     if response['Response'] == 'True':
#         try:
#             omdb_df.loc[index, 'Awards'] = response['Awards']
#             omdb_df.loc[index, 'Metascore'] = response['Metascore']
#             omdb_df.loc[index, 'IMDB'] = response['imdbRating']
#             omdb_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
#             omdb_df.loc[index, 'Rated'] = response['Rated']
#             omdb_df.loc[index, 'Director'] = response['Director']
#             omdb_df.loc[index, 'Runtime'] = response['Runtime']
#             omdb_df.loc[index, 'Country'] = response['Country']
#         except:
#             print(f'{row.Query_Title.upper()} (row {count}) has missing data')
#         count += 1
#     else:
#         print(f'{row.Query_Title.upper()} (row {count}) was not found')
#         count += 1

In [ ]:
# print(omdb_df.shape)
# omdb_df.head(1)

In [ ]:
# # Saving what we have so far
# omdb_df.to_csv('DataFiles/First_API_Run_BETA.csv', index=False)

# Loading for demo purposes
# omdb_df = pd.read_csv('DataFiles/First_API_Run_BETA.csv')
# omdb_df.head(5)

In [ ]:
# Nicole's code ends here

In [ ]:

# NICOLE'S CLEAN-UP CODE STARTS HERE


In [ ]:
# # Creating dataframe to hold subset request data
# omdb_successes_df = top_40_df.copy()
# omdb_successes_df['Awards'] = ''
# omdb_successes_df['Metascore'] = ''
# omdb_successes_df['IMDB'] = ''
# omdb_successes_df['Rotten Tomatoes'] = ''
# omdb_successes_df['Rated'] = ''
# omdb_successes_df['Director'] = ''
# omdb_successes_df['Runtime'] = ''
# omdb_successes_df['Country'] = ''

In [ ]:
# # ***** API requests All Data *****

# params = {"type": "movie", "apikey": OMB_api_key}
# url = "http://www.omdbapi.com/?t=&y="
# count = 0
# for index, row in omdb_successes_df.iterrows():
#     params["t"] = row["Query_Title"]
#     params["y"] = row["Year Released (Domestic)"]
#     response = requests.get(url, params).json()
#     if response['Response'] == 'True':
#         try:
#             omdb_successes_df.loc[index, 'Awards'] = response['Awards']
#             omdb_successes_df.loc[index, 'Metascore'] = response['Metascore']
#             omdb_successes_df.loc[index, 'IMDB'] = response['imdbRating']
#             omdb_successes_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
#             omdb_successes_df.loc[index, 'Rated'] = response['Rated']
#             omdb_successes_df.loc[index, 'Director'] = response['Director']
#             omdb_successes_df.loc[index, 'Runtime'] = response['Runtime']
#             omdb_successes_df.loc[index, 'Country'] = response['Country']
#         except:
#             omdb_successes_df = omdb_successes_df.drop(count)
#             print(f'{row.Query_Title.upper()} (row {count}) has missing data')
#     else:
#         print(f'{row.Query_Title.upper()} (row {count}) was not found')
#         omdb_successes_df = omdb_successes_df.drop(count)
#     count += 1

In [ ]:
# # Writing successes to file
# omdb_successes_df.to_csv('DataFiles/OMDB_Successes.csv')

In [ ]:
# Loading in Successes
omdb_successes_df = pd.read_csv('DataFiles/OMDB_Successes.csv')

In [ ]:
print(omdb_successes_df.shape)
omdb_successes_df.head(2)

In [ ]:
omdb_successes_df = omdb_successes_df.set_index('Unnamed: 0')

In [ ]:
print(omdb_successes_df.shape)
omdb_successes_df.head()

In [ ]:
omdb_failures_df = top_40_df[top_40_df['Title'].isin(omdb_successes_df['Title'])==False]

In [ ]:
omdb_failures_df = omdb_failures_df.reset_index()

In [ ]:
print(omdb_failures_df.shape)
omdb_failures_df.head()

In [ ]:
# Creating identical columns
omdb_failures_df = omdb_failures_df.copy()
omdb_failures_df['Awards'] = ''
omdb_failures_df['Metascore'] = ''
omdb_failures_df['IMDB'] = ''
omdb_failures_df['Rotten Tomatoes'] = ''
omdb_failures_df['Rated'] = ''
omdb_failures_df['Director'] = ''
omdb_failures_df['Runtime'] = ''
omdb_failures_df['Country'] = ''

In [ ]:
#overwrite cells with new query inputs
omdb_failures_df.at[0,'Query_Title'] = "DEAD MAN'S"
omdb_failures_df.at[1,'Query_Title'] = 'THE RISE OF SKYWALKER'
omdb_failures_df.at[2,'Query_Title'] = 'THE CURSE OF'
omdb_failures_df.at[3,'Query_Title'] = 'THE CHRONICLES OF NARNIA'
omdb_failures_df.at[4,"Query_Title"] = "PIRATES OF THE CARIBBEAN AT WORLD'S END"
omdb_failures_df.at[5,'Query_Title'] = 'HARRY POTTER AND THE ORDER OF THE PHOENIX'
omdb_failures_df.at[6,'Query_Title'] = 'THREE MEN AND A BABY'
omdb_failures_df.at[7,'Query_Title'] = 'MISSION IMPOSSIBLE II'
omdb_failures_df.at[8,'Query_Title'] = '9 To 5'
omdb_failures_df.at[9,'Query_Title'] = 'X-MEN'
omdb_failures_df.at[10,'Query_Title'] = 'MEN IN BLACK'
omdb_failures_df.at[11,'Query_Title'] = 'THE HOBBIT'
omdb_failures_df.at[12,'Query_Title'] = 'DUMB AND DUMBER'
omdb_failures_df.at[13,'Query_Title'] = 'THE GRINCH'
omdb_failures_df.at[14,'Query_Title'] = 'FAST & FURIOUS 6'
omdb_failures_df.at[15,'Query_Title'] = 'MR & MRS SMITH'
omdb_failures_df.at[16,'Query_Title'] = 'THE LORAX'
omdb_failures_df.at[17,'Query_Title'] = 'CROCODILE DUNDEE II'
omdb_failures_df.at[18,'Query_Title'] = 'INTERVIEW WITH THE VAMPIRE'
omdb_failures_df.at[19,'Query_Title'] = 'NIGHT AT THE MUSEUM BATTLE'
omdb_failures_df.at[20,'Query_Title'] = 'SPIDER-MAN INTO THE SPIDER-VERSE'
omdb_failures_df.at[21,'Query_Title'] = 'tt0089050'
omdb_failures_df.at[22,'Query_Title'] = 'DEAD MEN TELL NO TALES'
omdb_failures_df.at[23,'Query_Title'] = 'RISE OF THE SILVER SURFER'
omdb_failures_df.at[24,'Query_Title'] = 'FROM THE FILES OF POLICE SQUAD'
omdb_failures_df.at[25,'Query_Title'] = "A SERIES OF UNFORTUNATE EVENTS"
omdb_failures_df.at[26,'Query_Title'] = 'DODGEBALL'
omdb_failures_df.at[27,'Query_Title'] = 'A CHRISTMAS CAROL'
omdb_failures_df.at[28,'Query_Title'] = 'X-FILES'
omdb_failures_df.at[29,'Query_Title'] = 'FANTASTIC BEASTS'
omdb_failures_df.at[30,'Query_Title'] = 'I NOW PRONOUNCE YOU CHUCK & LARRY'
omdb_failures_df.at[31,'Query_Title'] = 'THREE MEN AND A LITTLE LADY'
omdb_failures_df.at[32,'Query_Title'] = 'tt0087355'
omdb_failures_df.at[33,'Query_Title'] = "CHEECH AND CHONG'S NEXT MOVIE"
omdb_failures_df.at[34,'Query_Title'] = 'INSURGENT'
omdb_failures_df.at[35,'Query_Title'] = 'LEGALLY BLONDE 2'
omdb_failures_df.at[36,'Query_Title'] = 'tt0113676'
omdb_failures_df.at[37,'Query_Title'] = 'ISLAND OF LOST DREAMS'
omdb_failures_df.at[38,'Query_Title'] = 'BLADE II'
omdb_failures_df.at[39,'Query_Title'] = 'ARTIFICIAL INTELLIGENCE'
omdb_failures_df.at[40,'Query_Title'] = 'THE HANGOVER PART III'
omdb_failures_df.at[41,'Query_Title'] = 'SPONGEBOB SQUAREPANTS MOVIE'
omdb_failures_df.at[42,'Query_Title'] = 'GREYSTOKE'
omdb_failures_df.at[43,'Query_Title'] = 'MOUSEHUNT'
omdb_failures_df.at[44,'Query_Title'] = 'NICE DREAMS'
omdb_failures_df.at[45,'Query_Title'] = 'tt0098484'
omdb_failures_df.at[46,'Query_Title'] = 'MAMMA MIA! HERE WE GO AGAIN'
omdb_failures_df.at[47,'Query_Title'] = 'EPIC'
omdb_failures_df.at[48,'Query_Title'] = 'FANTASIA 2000'
omdb_failures_df.at[49,'Query_Title'] = 'tt0370263'
omdb_failures_df.at[50,'Query_Title'] = 'FORD V FERRARI'
omdb_failures_df.at[51,'Query_Title'] = 'COWBOYS & ALIENS'
omdb_failures_df.at[52,'Query_Title'] = 'HIGH SCHOOL MUSICAL 3'
omdb_failures_df.at[53,'Query_Title'] = 'GNOMEO & JULIET'
omdb_failures_df.at[54,'Query_Title'] = 'FRIDAY THE 13TH PART III'
omdb_failures_df.at[55,'Query_Title'] = 'GARFIELD'
omdb_failures_df.at[56,'Query_Title'] = "MARCH OF THE PENGUINS"
omdb_failures_df.at[57,'Query_Title'] = 'A NIGHTMARE ON ELM STREET 4 THE DREAM MASTER'
omdb_failures_df.at[58,'Query_Title'] = 'DIVINE SECRETS OF THE YA-YA SISTERHOOD'
omdb_failures_df.at[59,'Query_Title'] = 'THE CONJURING 2'
omdb_failures_df.at[60,'Query_Title'] = 'AUSTIN POWERS INTERNATIONAL MAN OF MYSTERY'
omdb_failures_df.at[61,'Query_Title'] = 'Halloween H20: 20 Years Later'
omdb_failures_df.at[62,'Query_Title'] = 'KILL BILL Vol. 1'
omdb_failures_df.at[63,'Query_Title'] = 'PRINCE OF PERSIA: THE SANDS OF TIME'
omdb_failures_df.at[64,'Query_Title'] = 'A NIGHTMARE ON ELM STREET 3'
omdb_failures_df.at[65,'Query_Title'] = 'PERCY JACKSON & THE OLYMPIANS'
omdb_failures_df.at[66,'Query_Title'] = 'BARNYARD'
omdb_failures_df.at[67,'Query_Title'] = 'PLANES'
omdb_failures_df.at[68,'Query_Title'] = 'CITY SLICKERS II'
omdb_failures_df.at[69,'Query_Title'] = "DON'T BREATHE"
omdb_failures_df.at[70,'Query_Title'] = 'JOHN WICK CHAPTER 2'
omdb_failures_df.at[71,'Query_Title'] = 'FRIDAY THE 13TH: THE FINAL CHAPTER'
omdb_failures_df.at[72,'Query_Title'] = 'tt0080919'
omdb_failures_df.at[73,'Query_Title'] = "CAN'T BUY ME LOVE"
omdb_failures_df.at[74,'Query_Title'] = "A MADEA FAMILY FUNERAL"
omdb_failures_df.at[75,'Query_Title'] = 'tt0086352'
omdb_failures_df.at[76,'Query_Title'] = 'QUEST FOR FIRE'
omdb_failures_df.at[77,'Query_Title'] = 'tt0083628'
omdb_failures_df.at[78,'Query_Title'] = 'tt0081760'
omdb_failures_df.at[79,'Query_Title'] = 'tt0088885'
omdb_failures_df.at[80,'Query_Title'] = 'tt0081439'
omdb_failures_df.at[81,'Query_Title'] = 'FRIDAY THE 13TH A NEW BEGINNING'
omdb_failures_df.at[82,'Query_Title'] = "A NIGHTMARE ON ELM STREET 2 FREDDY'S REVENGE"

In [ ]:
# Running first set of failures
params = {"type": "movie", "apikey": OMB_api_key}
url = "http://www.omdbapi.com/?t=&y="
count = 0
for index, row in omdb_failures_df.iterrows():
    params["t"] = row["Query_Title"]
    params["y"] = row["Year Released (Domestic)"]
    response = requests.get(url, params).json()
    if response['Response'] == 'True':
        try:
            omdb_failures_df.loc[index, 'Awards'] = response['Awards']
            omdb_failures_df.loc[index, 'Metascore'] = response['Metascore']
            omdb_failures_df.loc[index, 'IMDB'] = response['imdbRating']
            omdb_failures_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
            omdb_failures_df.loc[index, 'Rated'] = response['Rated']
            omdb_failures_df.loc[index, 'Director'] = response['Director']
            omdb_failures_df.loc[index, 'Runtime'] = response['Runtime']
            omdb_failures_df.loc[index, 'Country'] = response['Country']
        except:
            omdb_failures_df = omdb_failures_df.drop(count)
            print(f'{row.Query_Title.upper()} has missing data')
    else:
        print(f'{row.Query_Title.upper()} was not found')
        omdb_failures_df = omdb_failures_df.drop(count)
    count += 1

In [ ]:
print(omdb_failures_df.shape)
omdb_failures_df.tail(50)
omdb_failures_df = omdb_failures_df.reset_index(drop=True)

In [ ]:
print(omdb_failures_df.shape)
omdb_failures_df.tail(1)

In [ ]:
# Writing failures to file
omdb_failures_df.to_csv('DataFiles/OMDB_Failures.csv')

In [ ]:
# Running second set of failures on imdb id - Couldn't get this to work so dumping

# TT0089050 was not found
# TT0087355 was not found
# TT0113676 was not found
# TT0098484 was not found
# TT0370263 was not found
# TT0080919 was not found
# TT0086352 was not found
# TT0083628 was not found
# TT0081760 was not found
# TT0088885 was not found
# TT0081439 was not found

# params = {"type": "movie", "apikey": OMB_api_key}
# url = "http://www.omdbapi.com/?i=&y="
# count = 0
# for index, row in omdb_failures2_df.iterrows():
#     params["i"] = row["Query_Title"]
#     params["y"] = row["Year Released (Domestic)"]
#     response = requests.get(url, params).json()
#     if response['Response'] == 'True':
#         try:
#             omdb_failures2_df.loc[index, 'OMDB Title'] = response['Title']
#             omdb_failures2_df.loc[index, 'Awards'] = response['Awards']
#             omdb_failures2_df.loc[index, 'Metascore'] = response['Metascore']
#             omdb_failures2_df.loc[index, 'IMDB'] = response['imdbRating']
#             omdb_failures2_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
#             omdb_failures2_df.loc[index, 'Rated'] = response['Rated']
#             omdb_failures2_df.loc[index, 'Director'] = response['Director']
#             omdb_failures2_df.loc[index, 'Runtime'] = response['Runtime']
#             omdb_failures2_df.loc[index, 'Country'] = response['Country']
#         except:
#             omdb_failures_df = omdb_failures2_df.drop(count)
#             print(f'{row.Query_Title.upper()} has missing data')
#     else:
#         print(f'{row.Query_Title.upper()} was not found')
#         omdb_failures_df = omdb_failures2_df.drop(count)
#     count += 1

In [ ]:
# Concatenating successes & failures
# OMDB_Final_df = pd.merge(omdb_failures_df, omdb_successes_df, on='')
frames = [omdb_failures_df, omdb_successes_df]
OMDB_Final_df = pd.concat(frames)

In [ ]:
print(OMDB_Final_df.shape)
OMDB_Final_df.head()

In [ ]:
OMDB_Final_df.to_csv('DataFiles/OMDB_Final.csv')

In [ ]:
# NICOLE'S CLEANUP CODE ENDS HERE

In [ ]:
# jason's code starts here

In [ ]:
# Creating 2 files one for data already populated (clean_test_df) for John to work with Analysis
# the second is for Marianne and I to parse through and find on OMDB
# clean_test_df = omdb_df.dropna()
# clean_test_df.head(50)
# clean_test_df.to_csv('DataFiles/clean_test_data.csv', index=False)

In [ ]:
# to_be_cleaned_df = omdb_df[pd.isnull(omdb_df['Awards'])]
# to_be_cleaned_df.head(50)
# to_be_cleaned_df.shape
# to_be_cleaned_df.to_csv('DataFiles/to_be_clean_data.csv', index=False)

In [ ]:
# to_be_clean_data = pd.read_csv('DataFiles/to_be_clean_data.csv')
# to_be_clean_data
# jason_cleanup_df= to_be_clean_data.head(75)
# jason_cleanup_df
#create a new dataframe with the movies that actually need cleaning
# j_cleaning_df = jason_cleanup_df[pd.isnull(jason_cleanup_df['Metascore'])&pd.isnull(jason_cleanup_df['IMDB'])]
# j_cleaning_df
# j_cleaning_df.shape

In [ ]:
# j_cleaning_df.at[[10],'Query_Title'] = "The battle of the five armies"
# j_cleaning_df

# j_cleaning_df.shape

In [ ]:
# pull out lines that were mistakenly included into the nan dataset based on awards stats
# and save for re-merging later 
# j_clean_awards_df = jason_cleanup_df.dropna(subset=['IMDB']) #'Metascore']), 'IMDB'])
# j_clean_awards_df
# j_clean_awards_df.shape

# jason_cleanup_df.to_csv('DataFiles/jason_cleanup_df.csv', index=False)

In [ ]:
# new_df = j_cleaning_df.append(j_clean_awards_df, ignore_index=True)
# new_df

In [ ]:
# new_df.to_csv('DataFiles/jason_cleanup_done_df.csv', index=False)

In [ ]:
#jason's code ends here

In [ ]:
# marianne's code starts here

In [ ]:
# take the last half of data to be cleaned up, based on 149 total rows of data
# cleaning_df = pd.read_csv('DataFiles/to_be_clean_data.csv')
# second_half_df = cleaning_df.tail(74)
# second_half_df

In [ ]:
# pull out lines that were mistakenly included into the nan dataset based on awards stats
# and save for re-merging later 

# m_clean_awards_df = second_half_df.dropna(subset=['IMDB'])
# m_clean_awards_df.shape

In [ ]:
#create a new dataframe with the movies that actually need cleaning
# m_cleaning_df = second_half_df[pd.isnull(second_half_df['IMDB'])]

# m_cleaning_df

In [ ]:
#overwrite cells with new query inputs
# m_cleaning_df.at[75,'Query_Title'] = 'Divine Secrets'
# m_cleaning_df.at[76,'Query_Title'] = 'conjuring 2'
# m_cleaning_df.at[78,'Query_Title'] = 'austin powers'
# m_cleaning_df.at[79,'Query_Title'] = 'Halloween H20'
# m_cleaning_df.at[80,'Query_Title'] = 'Kill Bill: Vol. 1'
# m_cleaning_df.at[82,'Query_Title'] = 'Prince of Persia'
# m_cleaning_df.at[85,'Query_Title'] = 'Nightmare On Elm Street 3'
# m_cleaning_df.at[87,'Query_Title'] = 'Percy Jackson & the Olympians'
# m_cleaning_df.at[90,'Query_Title'] = 'Barnyard'
# m_cleaning_df.at[91,'Query_Title'] = 'Planes'
# m_cleaning_df.at[92,'Query_Title'] = 'City Slickers II'
# m_cleaning_df.at[99,'Query_Title'] = "don't Breathe"
# m_cleaning_df.at[100,'Query_Title'] = 'John Wick Chapter 2'
# m_cleaning_df.at[104,'Query_Title'] = 'Friday the 13th: The Final Chapter'
# m_cleaning_df.at[124,'Query_Title'] = "can't buy me love"
# m_cleaning_df.at[125,'Query_Title'] = 'madea family funeral'
# m_cleaning_df.at[146,'Query_Title'] = 'A Nightmare on Elm Street 2'

# m_cleaning_df.head()


In [ ]:
#re-run the API for the updated query names
# params = {"type": "movie","apikey": OMB_api_key}
# url = "http://www.omdbapi.com/?t="
# count = 0
# for index, row in m_cleaning_df.iterrows():
#     params['t'] = row["Query_Title"]
#     params['y'] = row["Year Released (Domestic)"]
#     response = requests.get(url, params).json()
#     if response['Response'] == 'True':
#         try:
#             m_cleaning_df.loc[index, 'Awards'] = response['Awards']
#             m_cleaning_df.loc[index, 'Metascore'] = response['Metascore']
#             m_cleaning_df.loc[index, 'IMDB'] = response['imdbRating']
#             m_cleaning_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
#             m_cleaning_df.loc[index, 'Rated'] = response['Rated']
#             m_cleaning_df.loc[index, 'Director'] = response['Director']
#             m_cleaning_df.loc[index, 'Runtime'] = response['Runtime']
#             m_cleaning_df.loc[index, 'Country'] = response['Country']
#         except:
#             print(f'{row.Query_Title.upper()} (row {count}) has missing data')
#         count += 1
#     else:
#         print(f'{row.Query_Title.upper()} (row {count}) was not found')
#         count += 1

# m_cleaning_df.head()

In [ ]:
#concatenate both of marianne's dataframes back together

# frames = [m_clean_awards_df, m_cleaning_df]
# m_complete_df = pd.concat(frames)

# #save the completed data to a csv for later concatenation
# m_complete_df.to_csv('DataFiles/marianne_cleaned_data.csv', index=False)

In [ ]:
#import the previously saved csvs into dataframes

#import jason's data
# jason_df = pd.read_csv('DataFiles/jason_cleanup_done_df.csv')

# #import marianne's data
# marianne_df = pd.read_csv('DataFiles/marianne_cleaned_data.csv')

# #import the clean data separated earlier
# cleaned_df = pd.read_csv('DataFiles/clean_test_data.csv')

In [ ]:
# overwrite the remaining query titles not delivering results

# jason_df.at[34,'Query_Title'] = 'greystoke'
# jason_df.at[35,'Query_Title'] = 'mousehunt'
# jason_df.at[36,'Query_Title'] = 'nice dreams'
# jason_df.at[49,'Query_Title'] = 'mr & mrs smith'
# jason_df.at[65,'Query_Title'] = 'Blade II'
# jason_df.at[73,'Query_Title'] = 'Friday the 13th Part III'

In [ ]:
# run cleaned data through the API
# params = {"type": "movie","apikey": OMB_api_key}
# url = "http://www.omdbapi.com/?t="
# count = 0
# for index, row in jason_df.iterrows():
#     params['t'] = row["Query_Title"]
#     params['y'] = row["Year Released (Domestic)"]
#     response = requests.get(url, params).json()
#     if response['Response'] == 'True':
#         try:
#             jason_df.loc[index, 'Awards'] = response['Awards']
#             jason_df.loc[index, 'Metascore'] = response['Metascore']
#             jason_df.loc[index, 'IMDB'] = response['imdbRating']
#             jason_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
#             jason_df.loc[index, 'Rated'] = response['Rated']
#             jason_df.loc[index, 'Director'] = response['Director']
#             jason_df.loc[index, 'Runtime'] = response['Runtime']
#             jason_df.loc[index, 'Country'] = response['Country']
#         except:
#             print(f'{row.Query_Title.upper()} (row {count}) has missing data')
#         count += 1
#     else:
#         print(f'{row.Query_Title.upper()} (row {count}) was not found')
#         count += 1

In [ ]:
#concatenate marianne and jason's cleaned data with the clean data from earlier

# frames = [jason_df, marianne_df, cleaned_df]
# cleaned_query_titles_df = pd.concat(frames)

#output the cleaned query titles in a csv. 
#note: awards need to be cleaned next
# cleaned_query_titles_df.to_csv('DataFiles/queries_cleaned_all.csv', index=False)

In [ ]:
#import the csv to clean/update movie titles
# award_cleaning_df = pd.read_csv('DataFiles/queries_cleaned_all.csv')

In [ ]:
# award_cleaning_df.head()
# award_cleaning_df.shape

In [ ]:
#drop all movies with NaN in the awards column
# awardMovies_df = award_cleaning_df.dropna(subset=['Awards'])

# #insert column for Oscars Won
# awardMovies_df.insert(loc=11, column='Oscars Won', value="")

# # #limit to only Awards and Oscars Won columns
# awardMovies_df = awardMovies_df[['Title', 'Awards', 'Oscars Won']]

In [ ]:
# In the awards column, the description of oscar winners begins with the word "won"
#find oscar winners by finding the word 'won'
OMDB_Final_df.loc[OMDB_Final_df['Awards'].str.contains('Won', regex=False) == True, 'Oscars Won'] = 'Yes'
OMDB_Final_df.loc[OMDB_Final_df['Awards'].str.contains('Won', regex=False) == False, 'Oscars Won'] = 'No'

OMDB_Final_df.head()

In [ ]:
#filter down to movies that have won an oscar
oscar_df = OMDB_Final_df.loc[OMDB_Final_df['Oscars Won'] == 'Yes']
oscar_df.reset_index(drop=True, inplace=True)
print(oscar_df.shape)
oscar_df.head()

In [ ]:
#pull the number of oscars won, save in new column
for index, row in oscar_df.iterrows():
#     string = 
    oscar_df.loc[index, "Number Oscars Won"] = re.findall('\d+', oscar_df.loc[index, "Awards"])[0]

oscar_df

In [ ]:
#drop the Awards column from the oscar_df to avoid duplicating in the final dataframe
oscar_df = oscar_df[['Title', 'Year Released (Domestic)', 'Number Oscars Won']]

#re-merge the awards into the dataframe
FINAL_CLEANED_DF = pd.merge(OMDB_Final_df, oscar_df, 
                      how="left", on='Title')

FINAL_CLEANED_DF = pd.merge(OMDB_Final_df, oscar_df, 
                      how="left", on=['Title', 'Year Released (Domestic)'])

In [ ]:
#replace all NaNs in the number oscars won column with zero
FINAL_CLEANED_DF['Number Oscars Won'] = FINAL_CLEANED_DF['Number Oscars Won'].fillna(0)

#replace all NaNs in the oscars won column with 'no'
FINAL_CLEANED_DF['Oscars Won'] = FINAL_CLEANED_DF['Oscars Won'].fillna('No')

In [ ]:
# # Add Oscar percentage columns
# FINAL_CLEANED_DF['Percent Oscar Wins'] = FINAL_CLEANED_DF['Number Oscars Won'] / \
# FINAL_CLEANED_DF['Total Oscars Awarded in Year'] * 100

In [ ]:
print(FINAL_CLEANED_DF.shape)
FINAL_CLEANED_DF.tail(50)

In [ ]:
FINAL_CLEANED_DF = FINAL_CLEANED_DF[['Title', 'Domestic Release Date',
       'Year Released (Domestic)', 'Month Released (Domestic)',
       'Infl. Adj. Dom. Box Office', 'Domestic Box Office', 'Genre', 'Oscars Won', 'Number Oscars Won',
       'Total Oscars Awarded in Year', 'Awards',
       'Metascore', 'IMDB', 'Rotten Tomatoes', 'Rated', 'Director', 'Runtime',
       'Theatrical Distributor', 'Country']]

In [ ]:
FINAL_CLEANED_DF.to_csv('DataFiles/FINAL_CLEANED_DF.csv', index=False)

In [ ]:
FINAL_CLEANED_DF = pd.read_csv('DataFiles/FINAL_CLEANED_DF.csv')

In [ ]:
print(FINAL_CLEANED_DF.shape)
FINAL_CLEANED_DF.tail(5)

In [ ]:
# marianne's code ends here

In [ ]:
# NEXT STEPS
# Pull out NaN values from omdb_df and resave - DONE
# Create new dataframes with only NaN values & figure out how to make successful API calls on them - DONE
    # (might be a series of datframes & API calls after tweaking key words or maybe adding variable for year)
# Pull out oscar wins - DONE
# Save & review final dataframe - SAVED, ready for review with group